In [1]:
#Libraries import

import requests
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
import numpy as np

import os
import logging
import pandas as pd
!pip install scrapy
import scrapy
from scrapy.crawler import CrawlerProcess

import plotly.express as px
import plotly.io as pio

!pip install boto3
!pip install psycopg2-binary
import boto3
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# First Part - Data recovery

In [2]:
# Creation of a list containing the 35 cities for which we are going to get the GPS coordinates 

cities_list = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

# GPS coordinates

In [3]:
# Retrieving of latitude and longitude coordinates for each cities from the list
data_cities = [c.replace(' ','+') for c in cities_list]
gps_coord_lat = []
gps_coord_lon = []

In [4]:
#Loop to get all latitude and longitude for all cities
for city in data_cities:
    r = requests.get("https://nominatim.openstreetmap.org/search?q={}&country=france&format=json".format(city))
    gps_coord_lat.append(r.json()[0]['lat'])
    gps_coord_lon.append(r.json()[0]['lon'])

In [5]:
# Display all gps latitude coordinates 

print(gps_coord_lat)

['48.6359541', '48.649518', '49.2764624', '49.4938975', '49.4404591', '48.8588897', '49.8941708', '50.6365654', '48.584614', '48.249489800000006', '48.0777517', '48.0447968', '47.2380222', '47.3215806', '45.8992348', '45.1875602', '45.7578137', '43.7496562', '43.1506968', '43.2140359', '43.2961743', '43.5298424', '43.9492493', '44.0121279', '43.8374249', '43.5658225', '43.4522771', '42.52505', '43.2130358', '42.9455368', '43.6044622', '44.0175835', '43.471143749999996', '43.4945144', '46.1591126']


In [6]:
# Display all gps longitude coordinates


print(gps_coord_lon)

['-1.511459954959514', '-2.0260409', '-0.7024738', '0.1079732', '1.0939658', '2.3200410217200766', '2.2956951', '3.0635282', '7.7507127', '7.34429620253195', '7.3579641', '7.3079618', '6.0243622', '5.0414701', '6.1288847', '5.7357819', '4.8320114', '6.3285616', '6.3419285', '5.5396318', '5.3699525', '5.4474738', '4.8059012', '4.4196718', '4.3600687', '4.1912837', '4.4287172', '3.0831554', '2.3491069', '1.4065544156065486', '1.4442469', '1.3549991', '-1.552726590666314', '-1.4736657', '-1.1520434']


In [7]:
# Creation of a new dataframe containing gps coordinates regarding each cities

coord_list = [gps_coord_lat,gps_coord_lon]
df_coordinates = pd.DataFrame(coord_list,columns = cities_list, index = ('City_Lat', 'City_Lon'))
df_coordinates = df_coordinates.transpose()
df_coordinates['City'] = df_coordinates.index

In [8]:
# Display new dataframe

display(df_coordinates)

,City_Lat,City_Lon,City
Mont Saint Michel,48.6359541,-1.511459954959514,Mont Saint Michel
St Malo,48.649518,-2.0260409,St Malo
Bayeux,49.2764624,-0.7024738,Bayeux
Le Havre,49.4938975,0.1079732,Le Havre
Rouen,49.4404591,1.0939658,Rouen
Paris,48.8588897,2.3200410217200766,Paris
Amiens,49.8941708,2.2956951,Amiens
Lille,50.6365654,3.0635282,Lille
Strasbourg,48.584614,7.7507127,Strasbourg
Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195,Chateau du Haut Koenigsbourg


# Weather infos

In [9]:
# Lists creation in which data will be stored for the 4 chosen parameters

data_temp = []
data_weather = []
data_pop = []
data_wind = []

In [10]:
# Requesting API for chosen parameters

for c in range (35):
    d = requests.get('https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude=current,minutely,hourly,alerts&units=metric&appid=xxxxxxxxxxxxx'.format(gps_coord_lat[c],gps_coord_lon[c]))
    for day in range(1,8):
        data_temp.append(d.json()['daily'][day]['temp'])
        data_weather.append(d.json()['daily'][day]['weather'])
        data_pop.append(d.json()['daily'][day]['pop'])
        data_wind.append(d.json()['daily'][day]['wind_speed'])

In [11]:
# Retrieving data from brut infos

weather_forecast_first = []
temperature_forecast_first = []

# Retrieving data regarding text weather forecast and temperature forecast
for forecast in range(245):
    weather_forecast_first.append(data_weather[forecast][0]['description'])

temperature_forecast_first.append([d['day'] for d in data_temp]) 

In [12]:
# Sorting infos by city for 7 days

weather_forecast = []
rain_pop = []
temperature_forecast_first = temperature_forecast_first[0]
day_temp = []
wind_speed = []

weather_forecast = [weather_forecast_first[x:x+7] for x in range(0, len(weather_forecast_first),7)]
day_temp = [temperature_forecast_first[x:x+7] for x in range(0, len(temperature_forecast_first),7)]
rain_pop = [data_pop[x:x+7] for x in range(0, len(data_pop),7)]
wind_speed = [data_wind[x:x+7] for x in range(0, len(data_wind),7)]

In [13]:
# Gather all data in one dataframe

df_weather_infos = pd.DataFrame()
df_weather_infos['City'] = cities_list
df_weather_infos['7-Day Temperature'] = day_temp
df_weather_infos['7-Day Pop'] = rain_pop
df_weather_infos['7-Day Weather condition'] = weather_forecast
df_weather_infos['7-Day Wind speed'] = wind_speed

df_weather_infos

,City,7-Day Temperature,7-Day Pop,7-Day Weather condition,7-Day Wind speed
0,Mont Saint Michel,"[25.42, 28.2, 21.96, 21.78, 26.53, 28.86, 28.52]","[0.13, 0, 0.83, 0, 0, 0.08, 1]","[overcast clouds, few clouds, moderate rain, c...","[4.44, 6.69, 6.67, 6.69, 6.29, 8.36, 5.19]"
1,St Malo,"[23.48, 24.82, 19.9, 19.82, 24.44, 25.4, 23.37]","[0.2, 0, 0.49, 0, 0, 0.05, 1]","[light rain, few clouds, light rain, few cloud...","[4.85, 7, 7.08, 7.2, 7.49, 10.65, 6.31]"
2,Bayeux,"[25.99, 28.43, 21.11, 23.14, 21.37, 22.61, 26.66]","[0.24, 0, 0.85, 0, 0, 0, 0.99]","[light rain, broken clouds, light rain, few cl...","[4.25, 5.38, 4.63, 5, 7.05, 7.94, 7.52]"
3,Le Havre,"[22.08, 24.65, 20.95, 23.13, 23.62, 23.84, 25.56]","[0.3, 0, 0.61, 0.11, 0, 0, 0.98]","[light rain, overcast clouds, light rain, scat...","[4.53, 6.11, 5.12, 6.44, 8.4, 8.79, 9.05]"
4,Rouen,"[26.63, 30.52, 24.14, 27.51, 27.28, 27.39, 29.41]","[0, 0, 0.54, 0.69, 0.18, 0, 1]","[broken clouds, scattered clouds, light rain, ...","[4.54, 3.87, 5.65, 4.8, 5.11, 6.21, 5.57]"
5,Paris,"[28.56, 31.02, 33.04, 26.53, 28.13, 28.61, 29.67]","[0, 0, 0.03, 0.4, 0.27, 0, 0.92]","[broken clouds, broken clouds, overcast clouds...","[2.16, 2.69, 6.24, 3.95, 4.7, 4.91, 4.21]"
6,Amiens,"[26.94, 29.84, 30.83, 25.42, 25.69, 25.97, 28.66]","[0.05, 0, 0.41, 0.59, 0, 0, 1]","[scattered clouds, clear sky, light rain, ligh...","[5.67, 3.05, 5.28, 5.25, 6.32, 5.84, 6.1]"
7,Lille,"[26.83, 29.42, 28.73, 24.99, 25.31, 25.17, 26.94]","[0.02, 0, 0.11, 0.55, 0, 0, 0.69]","[scattered clouds, broken clouds, overcast clo...","[3.26, 3.48, 5.96, 5.67, 5.94, 6.38, 5.42]"
8,Strasbourg,"[28.82, 29.51, 30.12, 31.42, 26.04, 27.98, 31.02]","[0.03, 0, 0, 0, 1, 0.91, 0.73]","[scattered clouds, broken clouds, clear sky, f...","[2.25, 4.5, 2.13, 2.8, 4.41, 2.28, 2.18]"
9,Chateau du Haut Koenigsbourg,"[25.16, 25.37, 26.22, 27.47, 22.91, 24.89, 28.01]","[0, 0, 0, 0.13, 1, 0.87, 0.97]","[few clouds, broken clouds, clear sky, few clo...","[2.24, 3.81, 1.69, 2.51, 3.02, 1.64, 1.93]"


# Hotel infos

In [14]:
# Define the spider to crawl Booking pages and get the data that we want
class bookingSpider(scrapy.Spider):
    # Variables definition
    locs = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigue sMortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]
    name = "booking"
    init_url = dict() # set a dictionary for the urls to follow
    start_urls = ["https://www.booking.com/"]

    # Method to enter the cities in searching destination
    def parse(self, response):
        for loc in self.locs:
            # returns a new FormRequest object with its form field values pre-populated with those found in the HTML <form> element contained in the given response.
            yield scrapy.FormRequest.from_response(response,
                                                   formdata = {'ss': loc},
                                                   callback = self.after_search,
                                                   cb_kwargs = {'location':loc, 'page_no':0}
                                                   )

    # Method to find name, url, score and description of the hotels
    def after_search(self, response, location, page_no):
        # Condition to crawl pages
        if page_no == 0:
            self.init_url[location] = response.url
        
        cities = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for city in cities:
            name = city.css('div.fcab3ed991.a23c043802::text').get()
            url = city.css('a.e13098a59f').attrib['href']
            score = city.css('div.b5cd09854e.d10a6220b4::text').get()
            desc = city.css('div.d8eab2cf7f::text').get()
            
            dic = {'location': location,
                    'name': name,
                    'url': url,
                    'score':score,
                    'description': desc
                  }
            # initiate method for the parse_hotel GPS cordonnates 
            try: 
                yield response.follow(url = url,
                                      callback = self.GPS, 
                                      cb_kwargs = {'dic': dic}
                                     ) 
            except:
                dic['lat'] = None
                dic['lon'] = None
                yield dic
         
        if page_no <= 1: 
            # Create variable next_page by the url and end string to give to follow function below
            next_page = self.init_url[location]+"&offset="+str(25*(page_no+1)) 
            # follow method return a request instance to follow a link, here it is the next_page variable who contain previous url with a page tag at the end
            yield response.follow(next_page,
                                  callback = self.after_search,
                                  cb_kwargs = {'location':location, 'page_no':page_no+1}
                                 ) 

    # method to find GPS coordonates and split them in two variables lat & Lon        
    def GPS(self, response, dic): 
        try: 
            gps = response.css('a.jq_tooltip.loc_block_link_underline_fix.bui-link.show_on_map_hp_link.show_map_hp_link').attrib['data-atlas-latlng']   
        except:
            dic['lat']=None
            dic['lon']=None
            yield dic
            return None
        try:
            gps = gps.split(',')
            dic['lat']=float(gps[0])
            dic['lon']=float(gps[1])
            yield dic
        except:
            print("No way to split")
            dic['lat']=None
            dic['lon']=None
            yield dic
            
# Define a filename
filename = "scrap_booking.json"

# Remove previous file if it allready exist
if filename in os.listdir('hotels_scrapped/'):
        os.remove('hotels_scrapped/' + filename)

# Define the settings for the crawler process
process = CrawlerProcess(settings = {
    # simulate browser
    'USER_AGENT': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0', 
    # display regular errors
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'hotels_scrapped/' + filename : {"format": "json"},
    }
})

# Initiate crawler
process.crawl(bookingSpider)
process.start()

In [15]:
# Read the json file created by the spided in a dataframe

df_scrapping = pd.read_json('hotels_scrapped/scrap_booking.json')

In [16]:
# Printing shape of the new dataframe
df_scrapping.shape

(2618, 7)

In [17]:
# Checking missing infos and deleting lines with missing values

for column in df_scrapping:
    if df_scrapping[column].isnull().values.any():
        df_scrapping = df_scrapping[df_scrapping[column].isna() == False]

In [18]:
# Checking the new shape without missing values

df_scrapping.shape

(2397, 7)

In [19]:
# Printing first 5 rows of dataframe

df_scrapping.head()

,location,name,url,score,description,lat,lon
0,Amiens,The Originals Boutique Hôtel Amiens Sud,https://www.booking.com/hotel/fr/the-originals...,8.1,"Situated in Amiens, 1.9 km from Zénith d'Amien...",49.879613,2.266651
1,Paris,L'Empire Paris,https://www.booking.com/hotel/fr/empire-paris....,8.2,Located 300 metres from the Louvre Museum and ...,48.860638,2.342427
2,Strasbourg,Le Gutenberg,https://www.booking.com/hotel/fr/le-gutenberg....,8.6,Managed by a private host,48.581854,7.748158
3,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.3,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379
4,Besancon,ibis Styles Besançon,https://www.booking.com/hotel/fr/besancon.en-g...,7.8,"Conveniently located in a park, ibis Styles Be...",47.254253,6.021640


In [20]:
# Changing the name of the location column
df_scrapping.rename({'location': 'City'}, axis=1, inplace=True)

In [21]:
# Changing the name of the name column
df_scrapping.rename({'name': 'Hotel Name'}, axis=1, inplace=True)

In [22]:
# Merging all out dataframes
df_final = df_scrapping.merge(df_weather_infos, how = 'left' , on = 'City')
df_final = df_final.merge(df_coordinates, how = 'left' , on ='City')

In [23]:
# Checking final dataframe

df_final.head()

,City,Hotel Name,url,score,description,lat,lon,7-Day Temperature,7-Day Pop,7-Day Weather condition,7-Day Wind speed,City_Lat,City_Lon
0,Amiens,The Originals Boutique Hôtel Amiens Sud,https://www.booking.com/hotel/fr/the-originals...,8.1,"Situated in Amiens, 1.9 km from Zénith d'Amien...",49.879613,2.266651,"[26.94, 29.84, 30.83, 25.42, 25.69, 25.97, 28.66]","[0.05, 0, 0.41, 0.59, 0, 0, 1]","[scattered clouds, clear sky, light rain, ligh...","[5.67, 3.05, 5.28, 5.25, 6.32, 5.84, 6.1]",49.8941708,2.2956951
1,Paris,L'Empire Paris,https://www.booking.com/hotel/fr/empire-paris....,8.2,Located 300 metres from the Louvre Museum and ...,48.860638,2.342427,"[28.56, 31.02, 33.04, 26.53, 28.13, 28.61, 29.67]","[0, 0, 0.03, 0.4, 0.27, 0, 0.92]","[broken clouds, broken clouds, overcast clouds...","[2.16, 2.69, 6.24, 3.95, 4.7, 4.91, 4.21]",48.8588897,2.3200410217200766
2,Strasbourg,Le Gutenberg,https://www.booking.com/hotel/fr/le-gutenberg....,8.6,Managed by a private host,48.581854,7.748158,"[28.82, 29.51, 30.12, 31.42, 26.04, 27.98, 31.02]","[0.03, 0, 0, 0, 1, 0.91, 0.73]","[scattered clouds, broken clouds, clear sky, f...","[2.25, 4.5, 2.13, 2.8, 4.41, 2.28, 2.18]",48.584614,7.7507127
3,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.3,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379,"[25.42, 28.2, 21.96, 21.78, 26.53, 28.86, 28.52]","[0.13, 0, 0.83, 0, 0, 0.08, 1]","[overcast clouds, few clouds, moderate rain, c...","[4.44, 6.69, 6.67, 6.69, 6.29, 8.36, 5.19]",48.6359541,-1.511459954959514
4,Besancon,ibis Styles Besançon,https://www.booking.com/hotel/fr/besancon.en-g...,7.8,"Conveniently located in a park, ibis Styles Be...",47.254253,6.021640,"[27.2, 28.08, 29.4, 29.43, 28.81, 29.7, 31.26]","[0, 0, 0, 0.08, 0.38, 0.26, 0.99]","[clear sky, broken clouds, scattered clouds, s...","[3.24, 4.44, 2.11, 1.46, 3.15, 2.31, 5.61]",47.2380222,6.0243622


In [24]:
# Converting and saving dataframe in csv file

df_final.to_csv('Top_destinations_recommendations' , encoding='utf-8', index=False)

# Part 2 - Data lake & ETL

In [25]:
# Creating boto3 session
session = boto3.Session(aws_access_key_id='XXXXXXXXXXXXX', aws_secret_access_key='XXXXXXXXXXXXXXX')

In [26]:
# Selecting S3 bucket from AWS
s3 = session.resource("s3")

In [28]:
# Creating a s3 bucket 
bucket = s3.create_bucket(Bucket="kayak-recommendations-bucket", CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [29]:
# Uploading our csv file to the bucket
bucket.put_object(Key="Top_destinations_recommendations.csv", Body = 'Top_destinations_recommendations')

s3.Object(bucket_name='kayak-recommendations-bucket', key='Top_destinations_recommendations.csv')

In [30]:
# Connect to database

DBHOST = 'kayak-ar.ckedsinh979w.eu-west-3.rds.amazonaws.com'
DBUSER = 'XXXXXXXXXXXX'
DBPASS = 'XXXXXXXXXX'
DBNAME = 'postgres'

engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)

In [32]:
# Initialization of a sessionmaker to exchange with our database

Session = sessionmaker(bind = engine)

# Instanciate session 

session = Session()

df_final.to_sql('Top_destinations_recommendations_ds3', engine)

2022-08-22 15:26:42,450 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-08-22 15:26:42,451 INFO sqlalchemy.engine.Engine [cached since 75.26s ago] {'name': 'Top_destinations_recommendations_ds3'}
2022-08-22 15:26:42,477 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-22 15:26:42,478 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Top_destinations_recommendations_ds3" (
	index BIGINT, 
	"City" TEXT, 
	"Hotel Name" TEXT, 
	url TEXT, 
	score FLOAT(53), 
	description TEXT, 
	lat FLOAT(53), 
	lon FLOAT(53), 
	"7-Day Temperature" TEXT, 
	"7-Day Pop" TEXT, 
	"7-Day Weather condition" TEXT, 
	"7-Day Wind speed" TEXT, 
	"City_Lat" TEXT, 
	"City_Lon" TEXT
)


2022-08-22 15:26:42,479 INFO sqlalchemy.engine.Engine [no key 0.00082s] {}
2022-08-22 15:26:42,502 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Top_destinations_recommendations_ds3_index" ON "Top_destinations_

397

# Part 3 - Visualisation

In [33]:
# Getting the mean values of our selected parameters for all cities

df_final['Average Temperature'] = df_final['7-Day Temperature'].apply(np.mean)
df_final['Average Pop'] = df_final['7-Day Pop'].apply(np.mean)
df_final['Average Wind Speed'] = df_final['7-Day Wind speed'].apply(np.mean)

In [62]:
# Picking the top 5 destinations with the highest temperatures, the lowest probability of rain and the lowest wind speed

top_5_destinations = df_final.groupby('City').mean().sort_values('Average Temperature', ascending = True).head()
top_5_destinations = top_5_destinations.sort_values(['Average Pop', 'Average Wind Speed'], ascending = [False, False]).head()

In [63]:
display(top_5_destinations)

,score,lat,lon,Average Temperature,Average Pop,Average Wind Speed
City,,,,,,
Biarritz,8.068750,43.480068,-1.558651,24.574286,0.651429,4.545714
Ariege,8.361333,42.939334,1.526810,24.648571,0.541429,2.941429
Bayeux,8.868421,49.276302,-0.702956,24.187143,0.297143,5.967143
Le Havre,8.294737,49.494134,0.112269,23.404286,0.285714,6.920000
St Malo,8.786301,48.648228,-2.011885,23.032857,0.248571,7.225714


In [64]:
# Selecting the 20 best hotels in the top 5 cities

top_20_hotels = df_final[df_final['City'].isin(top_5_destinations.index)]
top_20_hotels = top_20_hotels.sort_values('score' , ascending= False).head(20)
display(top_20_hotels)

,City,Hotel Name,url,score,description,lat,lon,7-Day Temperature,7-Day Pop,7-Day Weather condition,7-Day Wind speed,City_Lat,City_Lon,Average Temperature,Average Pop,Average Wind Speed
1130,St Malo,"Appartement 4 chambres vue mer Intra-Muros, pa...",https://www.booking.com/hotel/fr/intra-muros-v...,10.0,Managed by a private host,48.646368,-2.025014,"[23.48, 24.82, 19.9, 19.82, 24.44, 25.4, 23.37]","[0.2, 0, 0.49, 0, 0, 0.05, 1]","[light rain, few clouds, light rain, few cloud...","[4.85, 7, 7.08, 7.2, 7.49, 10.65, 6.31]",48.649518,-2.0260409,23.032857,0.248571,7.225714
897,Le Havre,Les Appartements Océane Beau 2 pièces Quartier...,https://www.booking.com/hotel/fr/appartement-b...,10.0,Managed by a private host,49.492388,0.139885,"[22.08, 24.65, 20.95, 23.13, 23.62, 23.84, 25.56]","[0.3, 0, 0.61, 0.11, 0, 0, 0.98]","[light rain, overcast clouds, light rain, scat...","[4.53, 6.11, 5.12, 6.44, 8.4, 8.79, 9.05]",49.4938975,0.1079732,23.404286,0.285714,6.920000
1145,St Malo,Apartment Le Grand Large,https://www.booking.com/hotel/fr/apartment-st-...,10.0,"Offering sea views, Apartment Le Grand Large i...",48.652153,-2.017182,"[23.48, 24.82, 19.9, 19.82, 24.44, 25.4, 23.37]","[0.2, 0, 0.49, 0, 0, 0.05, 1]","[light rain, few clouds, light rain, few cloud...","[4.85, 7, 7.08, 7.2, 7.49, 10.65, 6.31]",48.649518,-2.0260409,23.032857,0.248571,7.225714
1155,St Malo,L'ILE AUX MOUETTES - en plein Intra-Muros,https://www.booking.com/hotel/fr/ile-aux-mouet...,10.0,"Located in Saint Malo, 300 metres from Cale de...",48.648786,-2.025717,"[23.48, 24.82, 19.9, 19.82, 24.44, 25.4, 23.37]","[0.2, 0, 0.49, 0, 0, 0.05, 1]","[light rain, few clouds, light rain, few cloud...","[4.85, 7, 7.08, 7.2, 7.49, 10.65, 6.31]",48.649518,-2.0260409,23.032857,0.248571,7.225714
1165,Bayeux,Modern apartment in Bayeux near Tapestry Museum,https://www.booking.com/hotel/fr/modern-apartm...,10.0,Modern apartment in Bayeux near Tapestry Museu...,49.276201,-0.699269,"[25.99, 28.43, 21.11, 23.14, 21.37, 22.61, 26.66]","[0.24, 0, 0.85, 0, 0, 0, 0.99]","[light rain, broken clouds, light rain, few cl...","[4.25, 5.38, 4.63, 5, 7.05, 7.94, 7.52]",49.2764624,-0.7024738,24.187143,0.297143,5.967143
924,Le Havre,The Four Seasons Panoramic View,https://www.booking.com/hotel/fr/the-four-seas...,10.0,"Offering free WiFi and city views, The Four Se...",49.489686,0.108133,"[22.08, 24.65, 20.95, 23.13, 23.62, 23.84, 25.56]","[0.3, 0, 0.61, 0.11, 0, 0, 0.98]","[light rain, overcast clouds, light rain, scat...","[4.53, 6.11, 5.12, 6.44, 8.4, 8.79, 9.05]",49.4938975,0.1079732,23.404286,0.285714,6.920000
1167,Bayeux,SAMANDINE HOME,https://www.booking.com/hotel/fr/samandine-hom...,10.0,Managed by a private host,49.276643,-0.702284,"[25.99, 28.43, 21.11, 23.14, 21.37, 22.61, 26.66]","[0.24, 0, 0.85, 0, 0, 0, 0.99]","[light rain, broken clouds, light rain, few cl...","[4.25, 5.38, 4.63, 5, 7.05, 7.94, 7.52]",49.2764624,-0.7024738,24.187143,0.297143,5.967143
1141,St Malo,L'ILOT,https://www.booking.com/hotel/fr/l-39-ilot.en-...,9.9,Managed by a private host,48.661135,-1.987553,"[23.48, 24.82, 19.9, 19.82, 24.44, 25.4, 23.37]","[0.2, 0, 0.49, 0, 0, 0.05, 1]","[light rain, few clouds, light rain, few cloud...","[4.85, 7, 7.08, 7.2, 7.49, 10.65, 6.31]",48.649518,-2.0260409,23.032857,0.248571,7.225714
1182,Bayeux,Le Chat Qui Veille,https://www.booking.com/hotel/fr/gite-6-47-9-p...,9.8,Managed by a private host,49.284642,-0.714551,"[25.99, 28.43, 21.11, 23.14, 21.37, 22.61, 26.66]","[0.24, 0, 0.85, 0, 0, 0, 0.99]","[light rain, broken clouds, light rain, few cl...","[4.25, 5.38, 4.63, 5, 7.05, 7.94, 7.52]",49.2764624,-0.7024738,24.187143,0.297143,5.967143
1451,Ariege,Camping jardin La Vie en Vert en Ariège,https://www.booking.com/hotel/fr/camping-la-vi...,9.8,"Located in Augirein, 42 km from Espot, Camping...",42.931479,0.919589,"[22, 27.3, 23.02, 17.98, 23.92, 29.72, 28.6]","[0.07, 0.63, 1, 1, 0.57, 0.34, 0.18]","[overcast clouds, moderate rain, light rain, l...","[2.71, 3.78, 2.42, 2.91, 3.08, 3.17, 

In [65]:
# Display weather map for the top 5 destinations
fig = px.scatter_mapbox(top_5_destinations,
                        title='Top 5 destinations weather map',
                        lat = 'lat',
                        lon = 'lon',
                        color = 'Average Temperature',
                        color_continuous_scale = px.colors.diverging.balance,
                        size = 'Average Wind Speed',
                        size_max = 30,
                        )
fig.update_layout(width = 1000,
                  height = 800,
                  title_x = 0.5, 
                  mapbox = {"style": "carto-darkmatter", "center": {"lon": 2, "lat" : 47}, "zoom": 4.8},
                  margin = {"l": 0, "r": 0, "b": 0, "t": 80},
                  )

fig.show()

In [66]:
# Display top 20 hotels for the top 5 destinations
fig = px.scatter_mapbox(top_20_hotels,
                        title = 'Top 20 hotels for the top 5 destinations',
                        lat = 'lat',
                        lon = 'lon',
                        color = 'score',
                        size = 'score',
                        color_continuous_scale = px.colors.diverging.Tropic,
                        size_max = 20,
                        )
fig.update_layout(width = 1000,
                  height = 800,
                  title_x = 0.5, 
                  mapbox = {"style": "carto-darkmatter", "center": {"lon": 2.6, "lat" : 43.5}, "zoom": 7},
                  margin = {"l": 0, "r": 0, "b": 0, "t": 80},
                  )
fig.show()